# Scan in $(hkl)$ coordinates

This notebook demonstrates how to scan in $(hkl)$ coordinates. It uses the
simulated 4-circle geometry from the `"hkl_soleil"` solver. The wavelength and
sample are whatever the simulator provides as defaults.

**Important**:  It is possible to scan in any combination of reciprocal axes or to
scan in any combination of real-space axes.  You are not allowed to scan in a
mix of reciprocal and real-space axes.

## Setup

First, create the simulated 4-circle diffractometer object (`e4cv`).

In [1]:
from hklpy2 import SimulatedE4CV

e4cv = SimulatedE4CV("", name="e4cv")

Setup Bluesky for running the scans with the `RE` object.  The `bec` object will
show a table of the data collected for each scan.  

For this simple demonstration, we won't add a databroker catalog.

In [2]:
from bluesky import RunEngine, plans as bp
from bluesky.callbacks.best_effort import BestEffortCallback

bec = BestEffortCallback()
RE = RunEngine()
RE.subscribe(bec)
bec.disable_plots()

We'll **import a simulator** (ready to use) from the `ophyd` package as a noisy detector.

In [3]:
from ophyd.sim import noisy_det

## (h10) scan

Scan the (reciprocal space) $h$ axis from -0.5 to +0.5 with $k=1$ and $l=0$.
This is called an $(h10)$ scan.

<details>

The computation to convert reciprocal-space values $(h,k,l)$ into real-space
angles ($\omega$, $\chi$, $\phi$, $2\theta$) is called the `forward()`
transformation.  The transformation is not necessarily unique.  The most common
way to reduce the number of *solutions* is to tell the solver which `mode` to
use.  The `mode` adds an additional pre-designed rule that constrains the
acceptable solutions.  The solver's geometry (in this case `E4CV`) provides the
list of known modes.

Note: Even with a chosen mode, the solution might not be unique.  In such cases,
the first solution returned by the `forward()` transformation is chosen.  The
user can change this by providing a different function for the diffractometer's
`_forward_solution` attribute.  The default is the
`hklpy2.diffract.pick_first_item()` function.

</details>

Here, the diffractometer starts with `"bissector"` mode (requires `tth = 2*omega`).

In [4]:
print(f"{e4cv.operator.solver.mode=!r}")
e4cv.k.move(1)
e4cv.l.move(0)
RE(bp.scan([noisy_det], e4cv.h, -0.5, 0.5, 11))

e4cv.operator.solver.mode='bissector'


Transient Scan ID: 1     Time: 2024-06-22 09:52:43
Persistent Unique Scan ID: 'f23b87bf-aa46-4c22-996b-1d4c48f3ba01'
New stream: 'primary'
+-----------+------------+------------+------------+
|   seq_num |       time |     e4cv_h |  noisy_det |
+-----------+------------+------------+------------+
|         1 | 09:52:43.0 |     -0.500 |      0.923 |
|         2 | 09:52:43.1 |     -0.400 |      1.033 |
|         3 | 09:52:43.1 |     -0.300 |      0.920 |
|         4 | 09:52:43.1 |     -0.200 |      0.987 |
|         5 | 09:52:43.1 |     -0.100 |      1.033 |
|         6 | 09:52:43.1 |      0.000 |      0.948 |
|         7 | 09:52:43.1 |      0.100 |      1.028 |
|         8 | 09:52:43.1 |      0.200 |      1.040 |
|         9 | 09:52:43.1 |      0.300 |      1.056 |
|        10 | 09:52:43.1 |      0.400 |      1.039 |
|        11 | 09:52:43.1 |      0.500 |      0.927 |
+-----------+------------+------------+------------+
generator scan ['f23b87bf'

('f23b87bf-aa46-4c22-996b-1d4c48f3ba01',)

**Clearly we see** that $h$ has been stepped across the range of -0.5 to +0.5.
Values for the noisy detector have been reported at each step.  But we want to
know about *all* the $hkl$ and angle values so we can observe the effects of
`"bissector"` mode.

### Scan again, showing all $(hkl)$ and real-space axes

Repeat the scan, same as before with a slight variation.  This time, add the
`e4cv` object as an additional detector.

In [5]:
print(f"{e4cv.operator.solver.mode=!r}")
e4cv.k.move(1)
e4cv.l.move(0)
RE(bp.scan([noisy_det, e4cv], e4cv.h, -0.5, 0.5, 11))

e4cv.operator.solver.mode='bissector'


Transient Scan ID: 2     Time: 2024-06-22 09:52:43
Persistent Unique Scan ID: 'ecb06edd-eeca-479b-ad2f-2fe1a2579a99'
New stream: 'primary'
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|   seq_num |       time |     e4cv_h |  noisy_det |     e4cv_k |     e4cv_l | e4cv_omega |   e4cv_chi |   e4cv_phi |   e4cv_tth |
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|         1 | 09:52:43.3 |     -0.500 |      1.030 |      1.000 |      0.000 |    -33.988 |    -63.435 |     90.000 |    -67.976 |
|         2 | 09:52:43.3 |     -0.400 |      0.996 |      1.000 |      0.000 |    -32.583 |    -68.199 |     90.000 |    -65.165 |
|         3 | 09:52:43.4 |     -0.300 |      1.072 |      1.000 |     -0.000 |    -31.468 |    -73.301 |     90.000 |    -62.935 |
|         4 | 09:52:43.4 |     -0.2

('ecb06edd-eeca-479b-ad2f-2fe1a2579a99',)

## What other modes are available?

In [6]:
e4cv.operator.solver.modes

['bissector',
 'constant_omega',
 'constant_chi',
 'constant_phi',
 'double_diffraction',
 'psi_constant']

## Scan $(h10)$ holding $\omega$ at -30 degrees

Set the mode to `"constant_omega"`, then set $\omega=-30$ degrees.

In [7]:
e4cv.operator.solver.mode = "constant_omega"
e4cv.omega.move(-30)
print(f"{e4cv.omega.position=!r}")

e4cv.omega.position=-30


**Run the scan again** with the same command.

In [8]:
print(f"{e4cv.operator.solver.mode=!r}")
e4cv.k.move(1)
e4cv.l.move(0)
RE(bp.scan([noisy_det, e4cv], e4cv.h, -0.5, 0.5, 11))

e4cv.operator.solver.mode='constant_omega'


Transient Scan ID: 3     Time: 2024-06-22 09:52:43
Persistent Unique Scan ID: '763a50a8-64c3-4fdf-a380-5acb17fdf909'
New stream: 'primary'
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|   seq_num |       time |     e4cv_h |  noisy_det |     e4cv_k |     e4cv_l | e4cv_omega |   e4cv_chi |   e4cv_phi |   e4cv_tth |
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|         1 | 09:52:43.6 |     -0.500 |      1.037 |      1.000 |      0.000 |    -30.000 |    -63.714 |     81.054 |    -67.976 |
|         2 | 09:52:43.6 |     -0.400 |      1.002 |      1.000 |     -0.000 |    -30.000 |    -68.345 |     83.031 |    -65.165 |
|         3 | 09:52:43.7 |     -0.300 |      0.979 |      1.000 |     -0.000 |    -30.000 |    -73.364 |     84.887 |    -62.935 |
|         4 | 09:52:43.7 |    

('763a50a8-64c3-4fdf-a380-5acb17fdf909',)

## Scan $(\bar{1}kl)$ holding $\omega$ at -30 degrees

Keep mode as `"constant_omega"` and $\omega=-30$.  Set $h=-1$ and scan $k$ & $l$.

In [9]:
e4cv.h.move(-1)
print(f"{e4cv.operator.solver.mode=!r}")
RE(bp.scan([noisy_det, e4cv], e4cv.k, 0.9, 1.1, e4cv.l, -0.6, -0.4, 11))

e4cv.operator.solver.mode='constant_omega'


Transient Scan ID: 4     Time: 2024-06-22 09:52:43
Persistent Unique Scan ID: 'a65729ed-4f8b-4722-a09d-974196cd9065'
New stream: 'primary'
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|   seq_num |       time |     e4cv_k |     e4cv_l |  noisy_det |     e4cv_h | e4cv_omega |   e4cv_chi |   e4cv_phi |   e4cv_tth |
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|         1 | 09:52:43.9 |      0.900 |     -0.600 |      0.954 |     -1.000 |    -30.000 |    -39.821 |     36.793 |    -94.876 |
|         2 | 09:52:44.0 |      0.920 |     -0.580 |      1.057 |     -1.000 |    -30.000 |    -40.796 |     37.124 |    -95.244 |
|         3 | 09:52:44.0 |      0.940 |     -0.560 |      1.025 |     -1.000 |    -30.000 |    -41.770 |     37.424 |    -95.659 |
|         4 | 09:52:44.0 |    

('a65729ed-4f8b-4722-a09d-974196cd9065',)

## Scan $(h10)$ holding $\psi$ at 25 degrees around $(100)$

Set the mode to `"psi_constant"`, then set $h_2=1, k_2=0, l_2=0$ & $\psi=25$ degrees.

TODO: What is $\psi$?  What is $(h_2, k_2, l_2)$?  Is enabled by solver yet?

In [10]:
e4cv.operator.solver.mode = "psi_constant"

# TODO: Can this be even easier?
extras = {
    "h2": 1,
    "k2": 0,
    "l2": 0,
    "psi": 25,
}
e4cv.operator.solver.extras = extras

In [11]:
print(f"{e4cv.operator.solver.mode=!r}")
print(f"{e4cv.operator.solver.extras=!r}")
RE(bp.scan([noisy_det, e4cv], e4cv.k, 0.9, 1.1, e4cv.l, -0.6, -0.4, 11))

e4cv.operator.solver.mode='psi_constant'
e4cv.operator.solver.extras={'h2': 1.0, 'k2': 0.0, 'l2': 0.0, 'psi': 25.0}


Transient Scan ID: 5     Time: 2024-06-22 09:52:44
Persistent Unique Scan ID: 'b908fbbc-9853-424e-8f00-3b3ba7bab9fb'
New stream: 'primary'
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|   seq_num |       time |     e4cv_k |     e4cv_l |  noisy_det |     e4cv_h | e4cv_omega |   e4cv_chi |   e4cv_phi |   e4cv_tth |
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|         1 | 09:52:44.3 |      0.900 |     -0.600 |      0.908 |     -1.000 |      3.258 |    -74.691 |    -18.768 |    -94.876 |
|         2 | 09:52:44.3 |      0.920 |     -0.580 |      1.057 |     -1.000 |      2.461 |    -76.033 |    -18.698 |    -95.244 |
|         3 | 09:52:44.3 |      0.940 |     -0.560 |      1.054 |     -1.000 |      1.63

('b908fbbc-9853-424e-8f00-3b3ba7bab9fb',)

## Scan $\psi$ around $(100)$ with sample oriented at $(101)$

Set the mode to `"psi_constant"`, then set $h_2=1, k_2=0, l_2=0$. Step scan
$\psi$ through desired range, setting before reporting at each step.


need custom plan

- loop through psi:
  - set psi via extras
  - solution = forward(1,0,1)
  - move to solution
  - read detectors


In [136]:
import numpy
from bluesky import plan_stubs as bps
from bluesky import preprocessors as bpp
from ophyd import Signal
from hklpy2 import SolverError

In [137]:
def scan_extra_parameter(
    dfrct,
    detectors:list = [],
    axis: str = None,  # name of extra parameter to be scanned
    start:float = None,
    finish:float = None,
    num: int = None,
    pseudos: dict = None,
    reals: dict = None,
    extras:dict = {},  # define all but the 'axis', these will remain constant
    md:dict=None,
):
    if pseudos is not None and reals is not None:
        raise SolverError("Cannot define both pseudos and reals.")
    if pseudos is None and reals is None:
        raise SolverError("Must define either pseudos or reals.")
    forwardTransformation = reals is None

    _md = {
        "diffractometer": {
            "name": dfrct.name,
            "solver": dfrct.operator.solver.name,
            "geometry": dfrct.operator.solver.geometry,
            "engine": dfrct.operator.solver.engine_name,
            "mode": dfrct.operator.solver.mode,
            "extra_axes": dfrct.operator.solver.extra_axis_names,
        },
        "axis": axis,
        "start": start,
        "finish": finish,
        "num": num,
        "pseudos": pseudos,
        "reals": reals,
        "extras": extras,
        "transformation": "forward" if forwardTransformation else "inverse"
    }
    _md.update(md or {})

    signal = Signal(name=axis, value=start)
    controls = detectors
    controls.append(dfrct)
    controls.append(signal)
    # TODO: controls.append(extras_device)  # TODO: need Device to report ALL extras
    controls = list(set(controls))

    @bpp.stage_decorator(detectors)
    @bpp.run_decorator(md=_md)
    def _inner():
        dfrct.operator.solver.extras = extras
        for value in numpy.linspace(start, finish, num=num):
            yield from bps.mv(signal, value)

            dfrct.operator.solver.extras = {axis: value}  # just the changing one
            if forwardTransformation:
                solution = dfrct.forward(pseudos)
                # TODO: Could provide a test run without the moves.
                reals = []  # convert to ophyd real positioner objects
                for k, v in solution._asdict().items():
                    reals.append(getattr(dfrct, k))
                    reals.append(v)
                yield from bps.mv(*reals)
            else:
                pass  # TODO: inverse

            # yield from bps.trigger(detectors)
            yield from bps.create("primary")
            for item in controls:
                yield from bps.read(item)
            yield from bps.save()

    return (yield from _inner())

In [150]:
print(f"{e4cv.operator.solver.mode=!r}")
RE(
    scan_extra_parameter(
        e4cv,
        detectors=[noisy_det],
        pseudos=(1, 0, 1),
        axis="psi",
        start=4,
        finish=44,
        num=11,
        extras=dict(h2=1, k2=0, l2=0),
    ),
)

e4cv.operator.solver.mode='psi_constant'


Transient Scan ID: 89     Time: 2024-06-22 11:11:20
Persistent Unique Scan ID: '54de771b-d955-43ae-acb5-a8e36f974751'
New stream: 'primary'
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|   seq_num |       time |        psi |  noisy_det |     e4cv_h |     e4cv_k |     e4cv_l | e4cv_omega |   e4cv_chi |   e4cv_phi |   e4cv_tth |
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|         1 | 11:11:20.4 |      4.000 |      1.004 |      1.000 |     -0.000 |      1.000 |   -135.000 |   -176.000 |    -45.000 |    -90.000 |
|         2 | 11:11:20.4 |      8.000 |      1.004 |      1.000 |      0.000 |      1.000 |   -135.000 |   -172.000 |    -45.000 |    -90.000 |
|         3 | 11:11:20.5 |     12.000 |      1.004 |      1.000 |     -0.000 |      1.000 |   -13

('54de771b-d955-43ae-acb5-a8e36f974751',)